# Simple IceDEF Model

This is a simplified version of the full icedef model for the purpose of allowing easier perception of the relationships between classes and also for testing new components.

In [1]:
import numpy as np

from scipy.interpolate import RegularGridInterpolator as RGI

import pandas as pd

In [2]:
class Iceberg():
    
    def __init__(self, T, Y, X, Vy, Vx):
        self.T = T
        self.Y = Y
        self.X = X
        self.Vy = Vy
        self.Vx = Vx

In [3]:
berg = Iceberg(0, 0, 0, 0, 0)

In [4]:
class Ocean():
    
    def __init__(self, tn, yn, xn):
        self.T = np.asarray([t for t in range(tn)])
        self.Y = np.asarray([y for y in range(yn)])
        self.X = np.asarray([x for x in range(xn)])
        self.Vcy = np.ones([tn,yn,xn])
        self.Vcx = np.ones([tn,yn,xn])
        self.iVcy = RGI((self.T, self.Y, self.X), self.Vcy)
        self.iVcx = RGI((self.T, self.Y, self.X), self.Vcx)

In [5]:
ocean = Ocean(10,10,10)

In [6]:
class Atm():
    
    def __init__(self, tn, yn, xn):
        self.T = [t for t in range(tn)]
        self.Y = [y for y in range(yn)]
        self.X = [x for x in range(xn)]
        self.Vay = np.ones([tn,yn,xn])
        self.Vax = np.ones([tn,yn,xn])
        self.iVay = RGI((self.T, self.Y, self.X), self.Vay)
        self.iVax = RGI((self.T, self.Y, self.X), self.Vax)

In [7]:
atm = Atm(10,10,10)

In [8]:
def drift(T, Y, X, Vcy, Vcx, Vay, Vax):

    Vy = (Vcy + Vay)/20
    Vx = (Vcx + Vax)/20
    
    return Vy, Vx

In [9]:
class DriftSimulation():
    
    def __init__(self, berg, ocean, atm, drift):
        
        self.berg = berg
        self.ocean = ocean
        self.atm = atm
        self.drift = drift
        
        self.ymin = min(self.ocean.Y[0], self.atm.Y[0])
        self.ymax = max(self.ocean.Y[-1], self.atm.Y[-1])
        self.xmin = min(self.ocean.X[0], self.atm.X[0])
        self.xmax = max(self.ocean.X[-1], self.atm.X[-1])
        
        self.dt = 0.1
        self.nt = 10
        
        self.history = pd.DataFrame(columns=['T', 'Y', 'X'])
        
    
    def interpolate(self, T, Y, X):
        
        Vcy = self.ocean.iVcy([T, Y, X])[0]
        Vcx = self.ocean.iVcx([T, Y, X])[0]
        Vay = self.atm.iVay([T, Y, X])[0]
        Vax = self.atm.iVax([T, Y, X])[0]
        
        return Vcy, Vcx, Vay, Vax
    
    
    def in_bounds(self, Y, X):
        
        if not self.xmin < X < self.xmax:
            print('Iceberg out of bounds')
            return False
        
        elif not self.ymin < Y < self.ymax:
            print('Iceberg out of bounds')
            return False
        
        else:
            return True
    
    def setup_timestepper(self, t0, y0, x0, vy0, vx0, nt):
        
        t = np.empty(nt+1)
        y = np.empty(nt+1)
        x = np.empty(nt+1)
        vy = np.empty(nt+1)
        vx = np.empty(nt+1)
        vcy = np.empty(nt+1)
        vcx = np.empty(nt+1)
        vay = np.empty(nt+1)
        vax = np.empty(nt+1)
        
        t[0] = t0
        y[0] = y0
        x[0] = x0
        vy[0] = vy0
        vx[0] = vx0
        
        vcy[0], vcx[0], vay[0], vax[0] = self.interpolate(t0, y0, x0)
        
        return t, y, x, vy, vx, vcx, vcy, vay, vax
    
    
    def euler(self, f, t0, y0, x0, vy0, vx0, dt, nt):

        t, y, x, vy, vx, vcx, vcy, vay, vax = self.setup_timestepper(0, y0, x0, vy0, vx0, nt)

        for i in range(nt):
            
            vy[i+1], vx[i+1] = f(t[i], y[i], x[i], vcy[i], vcx[i], vay[i], vax[i])

            t[i+1] = t[i] + dt
            y[i+1] = y[i] + vy[i+1] * dt
            x[i+1] = x[i] + vx[i+1] * dt
            
            if not self.in_bounds(y[i+1], x[i+1]):
                t = t[:i+1]
                y = y[:i+1]
                x = x[:i+1]
                break

            vcy[i+1], vcx[i+1], vay[i+1], vax[i+1] = self.interpolate(t[i], y[i], x[i])

        return t, y, x 
    
    
    def rk2(self, f, t0, y0, x0, vy0, vx0, dt, nt):

        t, y, x, vy, vx, vcx, vcy, vay, vax = self.setup_timestepper(0, y0, x0, vy0, vx0, nt)        
        
        for i in range(nt):
        
            vy[i+1], vx[i+1] = f(t[i], y[i], x[i], vcy[i], vcx[i], vay[i], vax[i])

            half_y = y[i] + 0.5*dt*vy[i+1]
            half_x = x[i] + 0.5*dt*vx[i+1]
            half_t = t[i] + 0.5*dt
            
            if not self.in_bounds(half_y, half_x):
                t = t[:i+1]
                y = y[:i+1]
                x = x[:i+1]
                break

            half_vcy, half_vcx, half_vay, half_vax = self.interpolate(half_t, half_y, half_x)

            half_vy, half_vx = f(half_t, half_y, half_x, half_vcy, half_vcx, half_vay, half_vax)

            y[i+1] = y[i] + dt*half_vy
            x[i+1] = x[i] + dt*half_vx
            t[i+1] = t[i] + dt

            if not self.in_bounds(y[i+1], x[i+1]):
                t = t[:i+1]
                y = y[:i+1]
                x = x[:i+1]
                break

            vcy[i+1], vcx[i+1], vay[i+1], vax[i+1] = self.interpolate(t[i+1], y[i+1], x[i+1])

        return t, y, x
    
    
    def run(self, save=True):
        
        f = self.drift
        t0 = self.berg.T
        y0 = self.berg.Y
        x0 = self.berg.X
        vy0 = self.berg.Vy
        vx0 = self.berg.Vx
        dt = self.dt
        tn = self.nt
        
        t, y, x = self.rk2(f, t0, y0, x0, vy0, vx0, dt, tn)
        
        if save:
            for i in range(len(t)):
                self.history.loc[len(self.history)] = [t[i], y[i], x[i]]
    
        return t, y, x
            

In [10]:
berg = Iceberg(0, 0, 0, 0, 0)
sim = DriftSimulation(berg, ocean, atm, drift)

In [11]:
sim.run()

(array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]),
 array([ 0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,
         0.09,  0.1 ]),
 array([ 0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,
         0.09,  0.1 ]))

In [12]:
sim.history

,T,Y,X
0,0.0,0.00,0.00
1,0.1,0.01,0.01
2,0.2,0.02,0.02
3,0.3,0.03,0.03
4,0.4,0.04,0.04
5,0.5,0.05,0.05
6,0.6,0.06,0.06
7,0.7,0.07,0.07
8,0.8,0.08,0.08
9,0.9,0.09,0.09
